In [ ]:
#install and setup cell
#%%capture

!pip install yahooquery

from yahooquery import Ticker
from yahooquery import Screener
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from pandas import to_datetime
from pandas import to_numeric
from tabulate import tabulate

In [4]:
#setup cell 2
screen = Screener()

In [5]:
day_gainers_data = screen.get_screeners('day_gainers')
day_gainers_dicts = day_gainers_data['day_gainers']['quotes']
day_gainers_symbols = [d['symbol'] for d in day_gainers_dicts]
day_gainers_symbols
day_gainers = Ticker(day_gainers_symbols)

day_losers_data = screen.get_screeners('day_losers')
day_losers_dicts = day_losers_data['day_losers']['quotes']
day_losers_symbols = [d['symbol'] for d in day_losers_dicts]
day_losers_symbols
day_losers = Ticker(day_losers_symbols)

day_gainers.summary_detail.keys()
day_losers.summary_detail.keys()

dict_keys(['FIVN', 'NFE', 'SATS', 'ELF', 'AQN', 'ZK', 'ARWR', 'PODD', 'SRPT', 'NICE', 'ALKT', 'SEE', 'VSAT', 'ALRM', 'AMN', 'MODG', 'CLSK', 'GTLS', 'CRNX', 'LFST', 'RIVN', 'RNG', 'BILI', 'RUN', 'ASTS'])

In [ ]:
day_gainers_history = day_gainers.history(start='2019-01-10',end='2024-01-10')
day_gainers_earnings= day_gainers.earning_history
day_gainers_recommendation = day_gainers.recommendation_trend
#print(day_gainers_history)
#print(day_gainers_earnings)
#print(day_gainers_recommendation)

In [7]:
day_losers_history = day_losers.history(start='2019-01-10',end='2024-01-10')
day_losers_earnings = day_losers.earning_history
day_losers_recommendation = day_losers.recommendation_trend
#print(day_losers_history)
#print(day_losers_earnings)
#print(day_losers_recommendation)

In [8]:
day_gainers_history['symbol'] = day_gainers_history.index.get_level_values(0)
day_gainers_history['date'] = to_datetime(day_gainers_history.index.get_level_values(1)).date
day_gainers_history.reset_index(drop=True, inplace=True)
day_gainers_history[["date", "symbol", "adjclose"]]

day_gainers_average_close = day_gainers_history.groupby("symbol")["adjclose"].mean().sort_values(ascending=False)
day_gainers_std_dev = day_gainers_history.groupby("symbol")["adjclose"].std().sort_values(ascending=False)

In [9]:
day_gainers_history["adjclose"].isnull().sum()
#checking for nulls

0

In [10]:
day_losers_history['symbol'] = day_losers_history.index.get_level_values(0)
day_losers_history['date'] = to_datetime(day_losers_history.index.get_level_values(1)).date
day_losers_history[["date", "symbol", "adjclose"]]
day_losers_history.reset_index(drop=True, inplace=True)

day_losers_average_close = day_losers_history.groupby("symbol")["adjclose"].mean().sort_values(ascending=False)
day_losers_std_dev = day_losers_history.groupby("symbol")["adjclose"].std().sort_values(ascending=False)

In [11]:
day_losers_history["adjclose"].isnull().sum()
#checking for Null values

0

In [12]:
print(day_gainers_earnings.dtypes)

maxAge              int64
epsActual          object
epsEstimate        object
epsDifference      object
surprisePercent    object
quarter            object
period             object
dtype: object


In [13]:
day_gainers_earnings['symbol'] = day_gainers_earnings.index.get_level_values(0)
day_gainers_earnings['date'] = to_datetime(day_gainers_earnings.index.get_level_values(1)).date
day_gainers_earnings['epsActual'] = to_numeric(day_gainers_earnings['epsActual'])
day_gainers_earnings.reset_index(drop=True, inplace=True)
day_gainers_earnings[["date", "symbol", "epsActual"]]

day_gainers_average_earnings = day_gainers_earnings.groupby("symbol")["epsActual"].mean().sort_values(ascending=False)


In [14]:
day_losers_earnings['symbol'] = day_losers_earnings.index.get_level_values(0)
day_losers_earnings['date'] = to_datetime(day_losers_earnings.index.get_level_values(1)).date
day_losers_earnings['epsActual'] = to_numeric(day_losers_earnings['epsActual'])
day_losers_earnings.reset_index(drop=True, inplace=True)
day_losers_earnings[["date", "symbol", "epsActual"]]

day_losers_average_earnings = day_losers_earnings.groupby("symbol")["epsActual"].mean().sort_values(ascending=False)

In [15]:
print(day_losers_std_dev)

symbol
RNG     108.295762
PODD     64.813035
GTLS     48.898117
NICE     46.622037
FIVN     43.672418
ELF      34.440578
BILI     32.697535
SRPT     27.404831
RIVN     26.434979
AMN      23.344178
ARWR     18.358668
VSAT     17.983408
RUN      17.808606
ALRM     14.983091
NFE      12.384822
SEE      10.489701
CLSK      9.663180
ALKT      7.755460
SATS      7.567300
MODG      5.528165
LFST      4.998066
CRNX      4.574349
ASTS      3.015195
AQN       2.406759
Name: adjclose, dtype: float64


In [16]:
day_gainers_recommendation_2 = day_gainers_recommendation.drop(index = [0,2,3],level = 1)
day_gainers_recommendation_3 = day_gainers_recommendation_2.drop(columns = 'period')

#print(day_gainers_recommendation_3)

day_losers_recommendation_2 = day_losers_recommendation.drop(index = [0,2,3],level = 1)
day_losers_recommendation_3 = day_losers_recommendation_2.drop(columns = 'period')

#print(day_losers_recommendation_3)

In [17]:
day_gainers_data = pd.concat([day_gainers_average_close, day_gainers_average_earnings, day_gainers_std_dev], axis=1, ignore_index=True)

day_gainers_final = day_gainers_data.rename(columns={0:'Average Adj Close',1:'Average EPS', 2:"Standard Deviation"})
print(tabulate(day_gainers_final,headers = ['Average Adj Close', 'Average EPS', 'Standard Deviation'], tablefmt='orgtbl'))


|       |   Average Adj Close |   Average EPS |   Standard Deviation |
|-------+---------------------+---------------+----------------------|
| EXPE  |           123.694   |        2.7125 |             32.8298  |
| AKAM  |            97.2889  |        1.635  |             13.8821  |
| U     |            75.3839  |       -0.56   |             45.2465  |
| ONTO  |            71.497   |        1.13   |             30.09    |
| BFH   |            59.3072  |        2.43   |             28.6546  |
| TTD   |            54.1631  |        0.3475 |             23.5152  |
| DOCN  |            46.0081  |        0.4475 |             19.7228  |
| DOCS  |            41.3379  |        0.26   |             17.3972  |
| G     |            39.7562  |        0.7575 |              5.52711 |
| CAVA  |            39.4525  |        0.1025 |              7.00115 |
| SLVM  |            39.4153  |        1.2675 |              7.20796 |
| STEP  |            29.4254  |        0.36   |              5.95583 |
| ACLX

In [18]:
day_losers_data = pd.concat([day_losers_average_close, day_losers_average_earnings, day_losers_std_dev], axis=1, ignore_index=True)

day_losers_final = day_losers_data.rename(columns={0:'Average Adj Close',1:'Average EPS', 2:"Standard Deviation"})
print(tabulate(day_losers_final,headers = ['Average Adj Close', 'Average EPS', 'Standard Deviation'], tablefmt='orgtbl'))

|      |   Average Adj Close |   Average EPS |   Standard Deviation |
|------+---------------------+---------------+----------------------|
| PODD |           222.542   |        0.8475 |             64.813   |
| NICE |           202.086   |        2.335  |             46.622   |
| RNG  |           156.09    |        0.855  |            108.296   |
| GTLS |           120.083   |        1.845  |             48.8981  |
| SRPT |           109.208   |        0.1125 |             27.4048  |
| FIVN |            99.9705  |        0.5325 |             43.6724  |
| AMN  |            80.8854  |        1.53   |             23.3442  |
| ALRM |            63.0225  |        0.565  |             14.9831  |
| VSAT |            47.6193  |       -2.0525 |             17.9834  |
| SEE  |            44.0209  |        0.815  |             10.4897  |
| ARWR |            43.6361  |       -1.165  |             18.3587  |
| ELF  |            39.3505  |        0.7975 |             34.4406  |
| BILI |            

In [19]:
day_gainers_criteria = (day_gainers_final['Average EPS']>0.2)&(day_gainers_final['Standard Deviation']<10)

day_gainers_criteria_2 = day_gainers_criteria.to_dict()

for key in list(day_gainers_criteria_2.keys()):
  if day_gainers_criteria_2[key] == False:
    del day_gainers_criteria_2[key]

print(type(day_gainers_criteria_2))

print(day_gainers_criteria_2)

day_gainers_symbols = list(day_gainers_criteria_2.keys())

print(day_gainers_symbols)

<class 'dict'>
{'G': True, 'SLVM': True, 'STEP': True, 'CARG': True, 'EVH': True}
['G', 'SLVM', 'STEP', 'CARG', 'EVH']


In [20]:
print(day_gainers_symbols)

['G', 'SLVM', 'STEP', 'CARG', 'EVH']


In [21]:
day_losers_criteria = (day_losers_final['Average EPS']>0.2)&(day_losers_final['Standard Deviation']<15)
day_losers_criteria_2 = day_losers_criteria.to_dict()

for key in list(day_losers_criteria_2.keys()):
  if day_losers_criteria_2[key] == False:
    del day_losers_criteria_2[key]

print(type(day_losers_criteria_2))

print(day_losers_criteria_2)

day_losers_symbols = list(day_losers_criteria_2.keys())

print(day_losers_symbols)


<class 'dict'>
{'ALRM': True, 'SEE': True, 'NFE': True}
['ALRM', 'SEE', 'NFE']


In [22]:
print(day_losers_symbols)


['ALRM', 'SEE', 'NFE']


In [23]:
stock_symbols = (day_gainers_symbols + day_losers_symbols)

print(type(stock_symbols))
print(stock_symbols)

<class 'list'>
['G', 'SLVM', 'STEP', 'CARG', 'EVH', 'ALRM', 'SEE', 'NFE']


In [24]:
stocks = Ticker(stock_symbols)

In [25]:
stocks_history = stocks.history(start='2019-01-10',end='2024-01-10')
stocks_stats= DataFrame(stocks.key_stats)

stocks_history["symbol"] = stocks_history.index.get_level_values(0)
stocks_history["date"] = to_datetime(stocks_history.index.get_level_values(1)).date
stocks_history.reset_index(drop=True, inplace=True)
stocks_history[["date", "symbol", "adjclose"]]

stocks_beta = stocks_stats.loc[['beta']]
stocks_beta_2 = stocks_beta.transpose()

stocks_avg_price = DataFrame(stocks_history.groupby("symbol")["adjclose"].mean().sort_values(ascending=False))
stocks_std_dev = DataFrame(stocks_history.groupby("symbol")["adjclose"].std().sort_values(ascending=False))
stocks_std_dev_2= stocks_std_dev.rename(columns={'adjclose':'std dev'})

#print(stocks_avg_price)
#print(stocks_std_dev_2)

#print(stocks_beta_2)

stocks_stats_data = pd.concat([stocks_avg_price, stocks_std_dev_2, stocks_beta_2], axis=1, ignore_index=True)
print(tabulate(stocks_stats_data,headers = ['Average Adj Close', 'Standard Deviation', 'Beta'], tablefmt='orgtbl'))

|      |   Average Adj Close |   Standard Deviation |    Beta |
|------+---------------------+----------------------+---------|
| ALRM |             63.0225 |             14.9831  | nan     |
| SEE  |             44.0209 |             10.4897  |   1.357 |
| G    |             39.7562 |              5.52711 | nan     |
| SLVM |             39.4153 |              7.20796 |   0.914 |
| STEP |             29.4254 |              5.95583 | nan     |
| NFE  |             27.7736 |             12.3848  |   1.473 |
| CARG |             27.1265 |              8.11814 | nan     |
| EVH  |             20.7659 |              9.59115 | nan     |


In [26]:
stocks_pivot = stocks_history.pivot(columns="symbol", values="adjclose", index="date")
stocks_pivot["date"] = stocks_pivot.index
stocks_pivot.reset_index(inplace=True, drop=True)
stocks_pivot


symbol,ALRM,CARG,EVH,G,NFE,SEE,SLVM,STEP,date
0,54.369999,36.080002,19.200001,26.065685,NaN,32.912437,NaN,NaN,2019-01-10
1,54.889999,36.020000,18.260000,26.338581,NaN,33.350185,NaN,NaN,2019-01-11
2,55.000000,35.340000,18.120001,26.159790,NaN,33.268101,NaN,NaN,2019-01-14
3,56.400002,36.340000,18.120001,26.639702,NaN,33.441376,NaN,NaN,2019-01-15
4,57.650002,36.380001,18.370001,26.865541,NaN,33.532574,NaN,NaN,2019-01-16
...,...,...,...,...,...,...,...,...,...
1253,61.150002,22.840000,31.650000,33.783115,36.478104,35.193974,48.250397,30.740917,2024-01-03
1254,61.799999,22.750000,32.689999,33.783115,36.011711,35.856701,47.746349,30.800072,2024-01-04
1255,61.189999,22.490000,32.360001,33.882214,36.110943,36.499653,47.410316,31.056410,2024-01-05
1256,62.000000,22.990000,32.700001,34.219154,36.130791,36.321606,48.546902,31.943737,2024-01-08


In [27]:
px.line(stocks_pivot, x="date", y=stock_symbols, title="Stock Prices")

In [28]:
#returns
#stocks_returns_daily = stocks_history.sort_values(['symbol','date']).reset_index(drop=True)
#stocks_returns_daily_2= DataFrame(stocks_returns_daily.groupby('symbol')['adjclose'].pct_change())
stocks_returns_daily = stocks_history.sort_values(['symbol','date']).reset_index(drop=True)
stocks_returns_daily['return']= stocks_returns_daily.groupby('symbol')['adjclose'].pct_change()

stocks_returns_daily_pivot = stocks_returns_daily.pivot(columns="symbol", values="return", index="date")
stocks_returns_daily_pivot["date"] = stocks_returns_daily_pivot.index
stocks_returns_daily_pivot.reset_index(inplace=True, drop=True)

#print(stocks_daily_returns_avg)
#print(stocks_daily_stddev)

px.line(stocks_returns_daily_pivot, x='date',y=stock_symbols,title="Daily Stock Returns")

#px.line()

In [29]:
stocks_daily_returns_avg = stocks_returns_daily.groupby('symbol')['return'].mean()
stocks_daily_stddev = stocks_returns_daily.groupby('symbol')['return'].std()

import math
SQRT = math.sqrt(252)

stocks_annual_return = stocks_daily_returns_avg.mul(252)
stocks_annual_stddev = stocks_daily_stddev.mul(SQRT)
print(stocks_annual_return)
print(stocks_annual_stddev)

symbol
ALRM    0.104752
CARG    0.062979
EVH     0.316277
G       0.099374
NFE     0.433303
SEE     0.083422
SLVM    0.415019
STEP    0.176299
Name: return, dtype: float64
symbol
ALRM    0.410783
CARG    0.555747
EVH     0.637868
G       0.298582
NFE     0.634581
SEE     0.363780
SLVM    0.476115
STEP    0.402311
Name: return, dtype: float64


In [30]:
stocks_table = pd.concat([stocks_annual_return, stocks_annual_stddev, stocks_beta_2], axis=1, ignore_index=True)
print(tabulate(stocks_table,headers = ['Anualized Return', 'Annualized Standard Deviation', 'Beta'], tablefmt='orgtbl'))

|      |   Anualized Return |   Annualized Standard Deviation |    Beta |
|------+--------------------+---------------------------------+---------|
| ALRM |          0.104752  |                        0.410783 | nan     |
| CARG |          0.062979  |                        0.555747 | nan     |
| EVH  |          0.316277  |                        0.637868 | nan     |
| G    |          0.0993745 |                        0.298582 | nan     |
| NFE  |          0.433303  |                        0.634581 |   1.473 |
| SEE  |          0.0834218 |                        0.36378  |   1.357 |
| SLVM |          0.415019  |                        0.476115 |   0.914 |
| STEP |          0.176299  |                        0.402311 | nan     |


In [31]:
def sharpe_ratio (stock_return, stock_stdev, risk_free_rate, sharpe_threshold_value):
  calc_sharpe_ratio = (stock_return - risk_free_rate)/stock_stdev
  buy_sharpe_ratio = calc_sharpe_ratio[calc_sharpe_ratio>sharpe_threshold_value]
  return buy_sharpe_ratio

risk_free_rate = .05
sharpe_threshold_value = .5

selected_value = sharpe_ratio(stocks_annual_return, stocks_annual_stddev, risk_free_rate, sharpe_threshold_value)

recommended_stocks = selected_value.index.tolist()

if not recommended_stocks:
    print("No stocks meet the criteria for buying.")
else:
    print("Buy These Stocks")
    for stock in recommended_stocks:
        print(f"Stock: {stock}, Sharpe: {selected_value[stock]}")

Buy These Stocks
Stock: NFE, Sharpe: 0.6040249174269186
Stock: SLVM, Sharpe: 0.7666624424530742


In [32]:
def calculate_sma(series, window=20):
    return series.rolling(window=window).mean()

def calculate_ema(series, window=20):
    return series.ewm(span=window, adjust=False).mean()

def calculate_rsi(series, window=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_bollinger_bands(series, window=20):
    sma = series.rolling(window=window).mean()
    std_dev = series.rolling(window=window).std()
    upper_band = sma + (std_dev * 2)
    lower_band = sma - (std_dev * 2)
    return upper_band, lower_band

In [33]:
if not recommended_stocks:
    print("No stocks meet the criteria for buying.")
else:
    all_dataframes = []

    for symbol in recommended_stocks:

        ticker_data = Ticker(symbol)
        historical_data = ticker_data.history(period="max")
        historical_data.reset_index(inplace=True)

        historical_data['symbol'] = symbol

        historical_data['SMA'] = calculate_sma(historical_data['close'])
        historical_data['EMA'] = calculate_ema(historical_data['close'])
        historical_data['RSI'] = calculate_rsi(historical_data['close'])
        upper_band, lower_band = calculate_bollinger_bands(historical_data['close'])
        historical_data['Upper_Band'] = upper_band
        historical_data['Lower_Band'] = lower_band
        historical_data['date'] = pd.to_datetime(historical_data['date'])

        all_dataframes.append(historical_data)

if all_dataframes:
        all_stocks_history = pd.concat(all_dataframes, ignore_index=True)

In [ ]:
def plot_stock_dashboard(symbol, data):
    stock_data = data[(data['symbol'] == symbol) & (data['date'] >= '2023-01-01')]

    fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                        vertical_spacing=0.02,
                        subplot_titles=(f'{symbol} Price', 'RSI', 'Bollinger Bands'),
                        row_heights=[0.5, 0.25, 0.25])

    fig.add_trace(go.Scatter(x=stock_data['date'], y=stock_data['close'], name='Close'), row=1, col=1)
    fig.add_trace(go.Scatter(x=stock_data['date'], y=stock_data['SMA'], name='SMA'), row=1, col=1)
    fig.add_trace(go.Scatter(x=stock_data['date'], y=stock_data['EMA'], name='EMA'), row=1, col=1)
    fig.add_trace(go.Scatter(x=stock_data['date'], y=stock_data['Upper_Band'], name='Upper Band'), row=3, col=1)
    fig.add_trace(go.Scatter(x=stock_data['date'], y=stock_data['Lower_Band'], name='Lower Band'), row=3, col=1)

    fig.add_trace(go.Scatter(x=stock_data['date'], y=stock_data['RSI'], name='RSI'), row=2, col=1)

    fig.update_layout(height=800, title_text=f"Dashboard for {symbol}", showlegend=True)

    fig.show()

for symbol in recommended_stocks:
    plot_stock_dashboard(symbol, all_stocks_history)